# Open Sim RL Training
#### Imports

In [1]:
# Dependencies
import numpy as np
import tensorflow as tf
# Environment
from osim.env import L2RunEnv as ENV # rename environment to be used for training

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Agent Class
#### Imports

In [2]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.processors import WhiteningNormalizerProcessor
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

Using TensorFlow backend.


#### Class
Reference: https://github.com/keras-rl/keras-rl/blob/master/examples/ddpg_mujoco.py

In [3]:
class Agent:
    def __init__(self,env):
        nb_actions = env.action_space.shape[0]
        
        self.env = env
        self.actor = self.build_actor(env)
        self.critic, action_input = self.build_critic(env)
        self.loss = self.build_loss()

        self.memory = SequentialMemory(limit=100000, window_length=1)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.1)
        self.agent = DDPGAgent(   nb_actions=nb_actions, actor=self.actor, 
                                  critic=self.critic, critic_action_input=action_input,
                                  memory=self.memory, nb_steps_warmup_critic=1000, 
                                  nb_steps_warmup_actor=1000,
                                  random_process=self.random_process, 
                                  gamma=.99, target_model_update=1e-3,
                                  processor=WhiteningNormalizerProcessor()  )
        self.agent.compile([Adam(lr=1e-4), Adam(lr=1e-3)], metrics=self.loss)

    def build_loss(self):
        return ['mse']

    def build_actor(self,env):
        nb_actions = env.action_space.shape[0]
        actor = Sequential()
        actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
        actor.add(Dense(400))
        actor.add(Activation('relu'))
        actor.add(Dense(300))
        actor.add(Activation('relu'))
        actor.add(Dense(nb_actions,
                        activation='tanh',
                        kernel_constraint=  keras.constraints.min_max_norm(
                                            min_value=0,
                                            max_value=nb_actions,
                                            axis=1) ) )
        actor.summary()

        inD = Input(shape=(1,) + env.observation_space.shape)
        out = actor(inD)

        return Model(inD,out)

    def build_critic(self,env):
        nb_actions = env.action_space.shape[0]
        action_input = Input(shape=(nb_actions,), name='action_input')
        observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
        flattened_observation = Flatten()(observation_input)
        x = Dense(400)(flattened_observation)
        x = Activation('relu')(x)
        x = Concatenate()([x, action_input])
        x = Dense(300)(x)
        x = Activation('relu')(x)
        x = Dense(1)(x)
        x = Activation('linear')(x)

        critic = Model(inputs=[action_input, observation_input], outputs=x)
        critic.summary()

        return critic, action_input
    
    def fit(self, **kwargs):
        return self.agent.fit(self.env,**kwargs)
    
    def test(self, **kwargs):
        return self.agent.test(self.env,**kwargs)
    
    def save_weights(self,filename='ddpg_{}_weights.h5f'):
        self.agent.save_weights(filename.format("opensim"), overwrite=True)
        
    def load_weights(self,filename='ddpg_{}_weights.h5f'):
        self.agent.load_weights(filename.format("opensim"))

### Environment Class

In [4]:
class TrainEnv(ENV):
    pass
# TODO: define virtual assistant forces on agent
# TODO: define search through easier environments
# TODO: make environment harder once the agent has trained for challenge

# Run Simulation
#### Environment

In [5]:
env = TrainEnv(visualize=False)
observation = env.reset( )

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


#### Agent

In [6]:
agent = Agent(env)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 41)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               16800     
_________________________________________________________________
activation_1 (Activation)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               120300    
_________________________________________________________________
activation_2 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 18)                5418      
Total params: 142,518
Trainable params: 142,518
Non-trainable params: 0
_________________________________________________________________
____

### Load previously trained weights

In [7]:
agent.load_weights( )

### Train new weights

In [ ]:
for i in range(100): # Train in smaller batches to allow for interuption
    print("\n\niteration:",i)
    agent.fit(nb_steps=2000, visualize=False, verbose=2)
    ## Always save new weights
    agent.save_weights( )



iteration: 0
Training for 2000 steps ...
  195/2000: episode: 1, duration: 5.798s, episode steps: 195, steps per second: 34, episode reward: -0.801, mean reward: -0.004 [-0.020, 0.004], mean action: 0.004 [-1.023, 0.857], mean observation: 0.073 [-4.036, 5.242], loss: --, mean_squared_error: --, mean_q: --
  381/2000: episode: 2, duration: 4.646s, episode steps: 186, steps per second: 40, episode reward: -0.789, mean reward: -0.004 [-0.020, 0.005], mean action: 0.046 [-0.945, 1.067], mean observation: 0.065 [-5.320, 6.739], loss: --, mean_squared_error: --, mean_q: --
  563/2000: episode: 3, duration: 4.741s, episode steps: 182, steps per second: 38, episode reward: 0.560, mean reward: 0.003 [-0.003, 0.011], mean action: 0.050 [-0.843, 0.993], mean observation: 0.135 [-5.071, 6.784], loss: --, mean_squared_error: --, mean_q: --
  732/2000: episode: 4, duration: 4.466s, episode steps: 169, steps per second: 38, episode reward: 0.536, mean reward: 0.003 [-0.002, 0.010], mean action: 0.

  660/2000: episode: 16, duration: 4.745s, episode steps: 43, steps per second: 9, episode reward: 0.174, mean reward: 0.004 [0.001, 0.007], mean action: 0.039 [-1.119, 1.145], mean observation: -0.074 [-19.306, 10.080], loss: --, mean_squared_error: --, mean_q: --
  702/2000: episode: 17, duration: 4.391s, episode steps: 42, steps per second: 10, episode reward: 0.167, mean reward: 0.004 [0.001, 0.006], mean action: 0.031 [-1.068, 1.106], mean observation: -0.080 [-19.472, 10.284], loss: --, mean_squared_error: --, mean_q: --
  744/2000: episode: 18, duration: 4.439s, episode steps: 42, steps per second: 9, episode reward: 0.172, mean reward: 0.004 [0.001, 0.006], mean action: 0.031 [-1.102, 1.088], mean observation: -0.079 [-19.139, 10.211], loss: --, mean_squared_error: --, mean_q: --
  784/2000: episode: 19, duration: 4.227s, episode steps: 40, steps per second: 9, episode reward: 0.161, mean reward: 0.004 [0.001, 0.007], mean action: 0.046 [-1.065, 1.067], mean observation: -0.090

 1433/2000: episode: 12, duration: 6.753s, episode steps: 124, steps per second: 18, episode reward: -0.926, mean reward: -0.007 [-0.022, 0.005], mean action: 0.119 [-1.120, 1.210], mean observation: -0.021 [-15.630, 15.024], loss: 0.000746, mean_squared_error: 0.001493, mean_q: 0.407273
 1560/2000: episode: 13, duration: 9.036s, episode steps: 127, steps per second: 14, episode reward: -0.779, mean reward: -0.006 [-0.019, 0.008], mean action: 0.166 [-1.154, 1.179], mean observation: -0.019 [-12.694, 11.112], loss: 0.000885, mean_squared_error: 0.001769, mean_q: 0.411115
 1684/2000: episode: 14, duration: 6.166s, episode steps: 124, steps per second: 20, episode reward: -0.799, mean reward: -0.006 [-0.020, 0.008], mean action: 0.133 [-1.128, 1.254], mean observation: 0.010 [-29.551, 11.779], loss: 0.001544, mean_squared_error: 0.003088, mean_q: 0.418433
 1819/2000: episode: 15, duration: 7.333s, episode steps: 135, steps per second: 18, episode reward: -0.801, mean reward: -0.006 [-0.0

  822/2000: episode: 11, duration: 8.563s, episode steps: 75, steps per second: 9, episode reward: 0.110, mean reward: 0.001 [-0.004, 0.009], mean action: 0.125 [-1.098, 1.155], mean observation: -0.003 [-19.890, 17.429], loss: --, mean_squared_error: --, mean_q: --
  890/2000: episode: 12, duration: 12.107s, episode steps: 68, steps per second: 6, episode reward: 0.144, mean reward: 0.002 [-0.003, 0.009], mean action: 0.064 [-1.214, 1.119], mean observation: 0.001 [-20.189, 17.549], loss: --, mean_squared_error: --, mean_q: --
  958/2000: episode: 13, duration: 13.114s, episode steps: 68, steps per second: 5, episode reward: 0.143, mean reward: 0.002 [-0.003, 0.009], mean action: 0.106 [-1.144, 1.235], mean observation: 0.004 [-19.343, 17.863], loss: --, mean_squared_error: --, mean_q: --
 1041/2000: episode: 14, duration: 13.366s, episode steps: 83, steps per second: 6, episode reward: 0.062, mean reward: 0.001 [-0.004, 0.009], mean action: 0.111 [-1.140, 1.135], mean observation: -0

 1949/2000: episode: 17, duration: 7.937s, episode steps: 134, steps per second: 17, episode reward: -0.742, mean reward: -0.006 [-0.020, 0.007], mean action: -0.040 [-1.117, 1.098], mean observation: 0.020 [-32.519, 15.308], loss: 0.000152, mean_squared_error: 0.000303, mean_q: 0.467680
done, took 237.125 seconds


iteration: 6
Training for 2000 steps ...
   94/2000: episode: 1, duration: 7.968s, episode steps: 94, steps per second: 12, episode reward: 0.272, mean reward: 0.003 [-0.001, 0.009], mean action: 0.073 [-1.144, 1.242], mean observation: 0.018 [-13.086, 17.499], loss: --, mean_squared_error: --, mean_q: --
  185/2000: episode: 2, duration: 6.643s, episode steps: 91, steps per second: 14, episode reward: 0.283, mean reward: 0.003 [-0.001, 0.009], mean action: 0.088 [-1.077, 1.181], mean observation: 0.019 [-15.033, 17.262], loss: --, mean_squared_error: --, mean_q: --
  284/2000: episode: 3, duration: 7.350s, episode steps: 99, steps per second: 13, episode reward: 0.224, mea

...

 1715/2000: episode: 20, duration: 3.662s, episode steps: 99, steps per second: 27, episode reward: 0.470, mean reward: 0.005 [0.000, 0.011], mean action: 0.190 [-1.141, 1.216], mean observation: 0.144 [-13.554, 20.860], loss: 0.000027, mean_squared_error: 0.000055, mean_q: 0.284888
 1808/2000: episode: 21, duration: 3.395s, episode steps: 93, steps per second: 27, episode reward: 0.468, mean reward: 0.005 [0.001, 0.010], mean action: 0.181 [-1.151, 1.182], mean observation: 0.144 [-13.328, 20.942], loss: 0.000093, mean_squared_error: 0.000186, mean_q: 0.286440
 1901/2000: episode: 22, duration: 3.404s, episode steps: 93, steps per second: 27, episode reward: 0.469, mean reward: 0.005 [0.001, 0.011], mean action: 0.220 [-1.085, 1.175], mean observation: 0.145 [-13.533, 20.931], loss: 0.000021, mean_squared_error: 0.000041, mean_q: 0.287751
 1999/2000: episode: 23, duration: 3.624s, episode steps: 98, steps per second: 27, episode reward: 0.474, mean reward: 0.005 [0.001, 0.010], mean a

  546/2000: episode: 6, duration: 2.554s, episode steps: 91, steps per second: 36, episode reward: 0.470, mean reward: 0.005 [0.001, 0.010], mean action: 0.079 [-1.164, 1.147], mean observation: 0.142 [-21.935, 21.388], loss: --, mean_squared_error: --, mean_q: --
  637/2000: episode: 7, duration: 2.549s, episode steps: 91, steps per second: 36, episode reward: 0.469, mean reward: 0.005 [0.001, 0.010], mean action: 0.089 [-1.236, 1.144], mean observation: 0.142 [-21.753, 21.351], loss: --, mean_squared_error: --, mean_q: --
  728/2000: episode: 8, duration: 2.617s, episode steps: 91, steps per second: 35, episode reward: 0.465, mean reward: 0.005 [0.001, 0.010], mean action: 0.101 [-1.119, 1.155], mean observation: 0.142 [-21.970, 21.105], loss: --, mean_squared_error: --, mean_q: --
  819/2000: episode: 9, duration: 2.583s, episode steps: 91, steps per second: 35, episode reward: 0.469, mean reward: 0.005 [0.001, 0.010], mean action: 0.092 [-1.115, 1.154], mean observation: 0.142 [-21

 1347/2000: episode: 15, duration: 3.532s, episode steps: 92, steps per second: 26, episode reward: 0.478, mean reward: 0.005 [0.000, 0.011], mean action: 0.217 [-1.175, 1.166], mean observation: 0.147 [-14.029, 16.063], loss: 0.000041, mean_squared_error: 0.000082, mean_q: 0.278389
 1438/2000: episode: 16, duration: 3.535s, episode steps: 91, steps per second: 26, episode reward: 0.470, mean reward: 0.005 [0.000, 0.010], mean action: 0.182 [-1.169, 1.140], mean observation: 0.147 [-14.804, 15.559], loss: 0.000030, mean_squared_error: 0.000061, mean_q: 0.282745
 1525/2000: episode: 17, duration: 3.321s, episode steps: 87, steps per second: 26, episode reward: 0.473, mean reward: 0.005 [-0.000, 0.012], mean action: 0.196 [-1.143, 1.201], mean observation: 0.148 [-8.949, 17.185], loss: 0.000100, mean_squared_error: 0.000200, mean_q: 0.275157
 1616/2000: episode: 18, duration: 3.502s, episode steps: 91, steps per second: 26, episode reward: 0.472, mean reward: 0.005 [0.000, 0.011], mean a

  187/2000: episode: 2, duration: 2.565s, episode steps: 93, steps per second: 36, episode reward: 0.480, mean reward: 0.005 [-0.001, 0.013], mean action: 0.151 [-1.161, 1.347], mean observation: 0.149 [-11.335, 14.030], loss: --, mean_squared_error: --, mean_q: --
  280/2000: episode: 3, duration: 2.570s, episode steps: 93, steps per second: 36, episode reward: 0.478, mean reward: 0.005 [-0.001, 0.013], mean action: 0.164 [-1.125, 1.201], mean observation: 0.147 [-12.677, 14.274], loss: --, mean_squared_error: --, mean_q: --
  372/2000: episode: 4, duration: 2.636s, episode steps: 92, steps per second: 35, episode reward: 0.480, mean reward: 0.005 [0.000, 0.012], mean action: 0.156 [-1.126, 1.144], mean observation: 0.149 [-11.152, 14.306], loss: --, mean_squared_error: --, mean_q: --
  463/2000: episode: 5, duration: 2.605s, episode steps: 91, steps per second: 35, episode reward: 0.481, mean reward: 0.005 [0.000, 0.012], mean action: 0.175 [-1.206, 1.212], mean observation: 0.148 [-

  449/2000: episode: 5, duration: 2.533s, episode steps: 90, steps per second: 36, episode reward: 0.479, mean reward: 0.005 [-0.001, 0.012], mean action: 0.185 [-1.138, 1.171], mean observation: 0.150 [-10.600, 14.127], loss: --, mean_squared_error: --, mean_q: --
  541/2000: episode: 6, duration: 2.662s, episode steps: 92, steps per second: 35, episode reward: 0.484, mean reward: 0.005 [-0.001, 0.012], mean action: 0.185 [-1.147, 1.228], mean observation: 0.152 [-11.228, 14.427], loss: --, mean_squared_error: --, mean_q: --
  631/2000: episode: 7, duration: 2.561s, episode steps: 90, steps per second: 35, episode reward: 0.481, mean reward: 0.005 [-0.001, 0.013], mean action: 0.213 [-1.121, 1.176], mean observation: 0.152 [-10.877, 14.299], loss: --, mean_squared_error: --, mean_q: --
  720/2000: episode: 8, duration: 2.473s, episode steps: 89, steps per second: 36, episode reward: 0.476, mean reward: 0.005 [-0.000, 0.012], mean action: 0.199 [-1.161, 1.149], mean observation: 0.150 

### Test Agent

In [ ]:
# Finally, evaluate our algorithm for 5 episodes.
agent.test(nb_episodes=5, visualize=True, nb_max_episode_steps=1000)